In [1]:
from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *
import matplotlib.pyplot as plt
import numpy as np

import time
import json

In [2]:
chip=CHIP(PS("192.168.1.10", 7,debug=1))
chip.set_device_cfg(0)

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: flt           	字节码: 09 00 00 0f ff
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 80
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 01 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_reset     	字节码: 0b 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: cim_ss        	字节码: 0d 00 00 00 01
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ser_para_sel  	字节码: 0e 00 00 00 01

指令完整字节码:  55aa010900000fff
指令完整字节码:  55aa010100000080
指令完整字节码:  55aa010100000100
指令完整字节码:  55aa010b00000001
指令完整字节码:  55aa010d00000001
指令完整字节码:  55aa010e00000001
Failed to send message:
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: adc0_in       	字节码: 03 00 00 a2 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 00 02
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: adc1_in       	字节码: 04 00 00 a2 00
模式: 1
	帧头:               	字节码: 55 aa 01


In [3]:
res = np.zeros((256,256))
# res[0] = np.ones((1,256))
res[128,128] = 1
res[32,64] = 1
chip.write_point2(device_array=res,write_voltage=0.1,tg=5,pulse_width=1,set_device=True)

模式: 4
	帧头:               	字节码: 55 aa 04
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 0c
	指令: pl_dac_v      	字节码: 03 00 00 00
	指令: pl_dac_v      	字节码: 03 01 00 00
	指令: pl_dac_v      	字节码: 03 02 00 00
	指令: pl_dac_v      	字节码: 03 03 00 00
	指令: pl_dac_v      	字节码: 03 04 00 00
	指令: pl_dac_v      	字节码: 03 05 00 00
	指令: pl_dac_v      	字节码: 03 06 00 00
	指令: pl_dac_v      	字节码: 03 07 00 00
	指令: pl_dac_v      	字节码: 03 08 00 00
	指令: pl_dac_v      	字节码: 03 09 00 00
	指令: pl_dac_v      	字节码: 03 0a 00 00
	指令: pl_dac_v      	字节码: 03 0b 00 00
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: ins_num       	字节码: 1c 00 00 00 0c
模式: 1
	帧头:               	字节码: 55 aa 01
	指令: fast_command_1	字节码: 01 00 00 80 00

指令完整字节码:  55aa040000000c03000000030100000302000003030000030400000305000003060000030700000308000003090000030a0000030b0000
指令完整字节码:  55aa011c0000000c
指令完整字节码:  55aa010100008000
Failed to send message:
模式: 1
	帧头:          	字节码: 55 aa 01
	指令: pulse_cyc	字节码: 1b 05 f5 e1 00

指令完整字节码:  55aa011b05f5e

In [ ]:
chip.set_op_mode(read=False,row=True)
chip.write2(row_index=[0],col_index=[i for i in range(0,256,2)],write_voltage=2,tg=5)


# chip.write2(row_index=[i for i in range(0,256,2)],col_index=[2],write_voltage=2,tg=5)

In [ ]:
chip.set_op_mode(read=True,row=True)
chip.read_crossbar2(row_index=[0,2,4,6],col_index=[i for i in range(256)],read_voltage=0.1,tg=5,gain = 1,row = True,)

In [ ]:
bank_data = [ i for i in range(3,31)]
chunks = [bank_data[i:i+4] for i in range(0, len(bank_data), 4)]
print(chunks)

In [ ]:
col_data = chip.get_bank_index_tia([i for i in range(256)])
read_batch = chip.tia_split(col_data,check_tia=True)

for i in read_batch:
    print(i)

In [ ]:

for row_index in row_index_list:
    row_tmp = []
    row_data = chip.get_bank_index_tia(row_index)
    row_bank = chip.bank_split(row_data)
    for i in row_bank:
        bank,index = chip.get_bank_index32(i)

In [5]:
def write2(self,row_index:list,col_index:list,write_voltage:float,tg:float = 5):
    """
        写器件, row_index为行索引, col_index为列索引
    """
    assert self.op_mode == "write","未设置为写模式。"

    self.write_voltage = write_voltage
    # print(self.get_setting_info())
    # ----------------------------------------------从行还是列去写
    if self.from_row:
        # 从行写
        row_bank_ins, col_bank_ins =  PL_ROW_BANK, PL_COL_BANK
        write_ins = PL_WRITE_ROW_PULSE
    else:
        # 从列读
        row_index, col_index = col_index, row_index
        row_bank_ins, col_bank_ins =  PL_COL_BANK, PL_ROW_BANK
        write_ins = PL_WRITE_COL_PULSE

    # ----------------------------------------------ins_ram,din_ram的地址
    ins_ram_start = 0
    din_ram_start = 0

    # ----------------------------------------------发送要配置的bank的数据进去
    res_row_bank,res_col_bank,_ = self.send_read_din_ram2([row_index],col_index,din_ram_start,False)

    # ----------------------------------------------准备指令序列
    ins_data = []#self.get_dac_ins2(v=write_voltage,tg=tg)                                         # 配置电压

    ins_data.append(CMD(PL_CIM_RESET))
    for bank,din_ram_pos in res_row_bank[0]:
        ins_data.append(CMD(row_bank_ins,command_data=CmdData(bank<<8|din_ram_pos)))            # 从din_ram的din_ram_pos位置取数据配置bank

    for bank,din_ram_pos in res_col_bank[0]:
        ins_data.append(CMD(col_bank_ins,command_data=CmdData(bank<<8|din_ram_pos)))            # 从din_ram的din_ram_pos位置取数据配置bank

    ins_data.append(CMD(write_ins))                                                             # 写脉冲

    num = len(ins_data)
    ins_data.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(num)))
    ins_data.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(ins_ram_start)))

    # ----------------------------------------------发送指令序列并执行
    pkts=Packet()
    pkts.append_single(ins_data,mode=4)
    pkts.append_single([CMD(INS_NUM,command_data=CmdData(num))],mode=1)
    pkts.append_single([CMD(FAST_COMMAND_1,command_data=CmdData(FAST_COMMAND1_CONF.cfg_ins_run))],mode=1)
    self.ps.send_packets(pkts)

In [8]:
def set_device(row,col,write_voltage,tg,pulse_width):
    # chip.set_op_mode2(read=False,row=True,clearv=False)
    # chip.set_pulse_width(pulse_width)
    write2(chip,row_index=[row],col_index=[col],write_voltage=write_voltage,tg=tg)

In [ ]:
chip.ps.debug=1
for i in range(256):
    print(f"----------------------------------------------------------第{i}列-----------------------------------------------------------")
    set_device(100,i,2,2.5,1)

In [ ]:
for i in range(65,103,2):
    print(i)

In [ ]:
chip.ps.debug=True
chip.set_op_mode2(read=True,row=True)
# chip.set_dac_read_V(v=0.1,tg=5)
# chip.set_tia_gain(3)
chip.set_op_mode2(read=True,row=True)

In [ ]:
# vres,cres = chip.read(row_index=[0],col_index=[i for i in range(256)],all_tia=False,tia_flag=True)
# print(vres)
chip.ps.debug=True
vres,cres = chip.read2(
    row_index=[i for i in range(0,32,2)],
    col_index=[i for i in range(256)],
    read_voltage=0.1,tg=5,
    check_tia=True,sum=True
)
for i in vres:
    print(i)
print(vres)

In [ ]:
vres,cres = chip.read(row_index=[i for i in range(256)],col_index=[1],all_tia=False,tia_flag=True)
print(vres)

vres,cres = chip.read2(row_index=[i for i in range(256)],col_index=[1],read_voltage=0.1,tg=5,all_tia=False,tia_flag=True)
print(vres)

In [ ]:
for i in range(7):
    print(chip.bank_to_num([i]))
    print(len(chip.bank_to_num([i,i+1])))

In [ ]:
chip.set_device_cfg(deviceType=0)
for i in range(16):
    print(chip.tia_to_num([i],row=False))

In [ ]:
import numpy as np

In [ ]:

device_array = np.ones((256,256))
row,col = device_array.shape
points = []
# 因为行/列bank都是间隔2, 所以为了避免更多的切行列bank, 尽量使得一个bank的挨在一起
for i_start in range(2):
    for i in range(i_start,row,2):
        col_index = [j for j in range(0,col,2) if device_array[i][j]>0.5] + [j for j in range(1,col,2) if device_array[i][j]>0.5]
        points += [(i,j) for j in col_index]  
for i in range(0,len(points),32):
    print(points[i:i+32])


In [3]:
chip.set_device_cfg(deviceType=1)

In [ ]:
chip.ps.debug = True
chip.set_op_mode2(read=True,row=False)
vres,_ = chip.read2(row_index=[i for i in range(256)],col_index=[0],read_voltage=0.1,tia_flag=True)
print(vres)

In [5]:
ps.debug = True

In [ ]:
cmd=[
    CMD(PL_DATA,command_data=CmdData(0x1234_5678)),
    CMD(PL_DATA,command_data=CmdData(0x8765_4321)),
]

cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=5)
ps.send_packets(pkts)

In [4]:
ps.debug = True
ps.enable = True

In [ ]:
cmd=[
    CMD(PL_ROW_BANK,command_data=CmdData(1<<(1+8) | 0)),
    CMD(PL_COL_BANK,command_data=CmdData(1<<(2+8) | 1)),
    CMD(PL_DAC_V,command_data=CmdData(12<<16 | 0xFFF0)),
    CMD(PL_WRITE_ROW_PULSE),
    CMD(PL_WRITE_COL_PULSE),
    CMD(PL_READ_ROW_PULSE,command_data=CmdData(0)),
    CMD(PL_READ_COL_PULSE,command_data=CmdData(1)),
]
num = len(cmd)
cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(num)))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=4)
pkts.append_cmdlist([
    CMD(INS_NUM,command_data=CmdData(num)),
    CMD(FAST_COMMAND_1,command_data=CmdData(FAST_COMMAND1_CONF.cfg_ins_run)),
],mode=1)

ps.send_packets(pkts)
ps.send_packets(pkts)

In [ ]:
cmd=[
    CMD(PL_RAM_ADDR,command_data=CmdData(0)),
    CMD(PL_DATA_LENGTH,command_data=CmdData(2))
    # CMD(PL_DAC_V,command_data=CmdData(dac_channel<<16 | voltage))
]

# cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
# cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=6)
ps.send_packets(pkts)



message=ps.receive_packet()
print(message.hex())

In [ ]:

row_bank=[[],[1]]
row_bank = [i for i in row_bank if len(i)>0]
print(row_bank)

In [ ]:
cmd=[
    CMD(PL_DATA,command_data=CmdData(0x1234_5678)),
    CMD(PL_DATA,command_data=CmdData(0x8765_4321)),
    # CMD(PL_DATA,command_data=CmdData(0xFFFF_FFFF)),
]

cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=5)
ps.send_packets(pkts)

In [ ]:
# chip.read_set(row=True,v=0.1,gain=3)
message_cond = []
message_v = []
for TIA_num in [i for i in range(16)]:
    adc_num, adc_channel = int(TIA_num/4),TIA_num%4

    channel_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    pkts=Packet()
    adc_out = CMD(dict(
        command_addr = 65+TIA_num,
        command_type = COMMAND_TYPE[1],
        n_addr_bytes = N_ADDR_BYTES[0],
        n_data_bytes = N_DATA_BYTES[0],
        command_name = f"adc{adc_num}_out{channel_map[adc_channel]}",
        command_data = CmdData(0),
        command_description = "从ADC对应通道读取数据"
    ))
    pkts.append_cmdlist([adc_out],mode=2)
    print(pkts)
    # 发送指令
    ps.send_packets(pkts)
    # 接收信息
    # 高16bit: 0, 低16bit: 寄存器的16bit值
    message = ps.receive_packet(f"adc读取:{adc_out.command_name}")
    print(message.hex())

In [ ]:
# bank = 1
# din_ram_add = 10

# dac_channel = 12

cmd=[
    CMD(PL_RAM_ADDR,command_data=CmdData(0)),
    CMD(PL_DATA_LENGTH,command_data=CmdData(1))
    # CMD(PL_DAC_V,command_data=CmdData(dac_channel<<16 | voltage))
]

# cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
# cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=6)
ps.send_packets(pkts)
print(pkts)


message=ps.receive_packet()
print(message.hex())

In [ ]:
print(message.hex()[0:4])
# adc通道 1, 0, 3, 2...... 

In [ ]:
test=CMD(DAC_IN,command_data=CmdData(1<<24|0x0400ff))

test = CMD(CIM_BANK_SEL,command_data=CmdData(1<<7))
print(bytes(test))


print(test.concatenate().to_bytes((test.n_addr_bytes + test.n_data_bytes), "little"))

In [ ]:
value = 0x1111_FFFF
big_endian_bytes = value.to_bytes(4, byteorder='big')

print(big_endian_bytes)

In [ ]:
# 用于和下位机交互通信
ps=PS(1,1,delay=10*1e-3,debug=False)
# 控制ADC
adc=ADC(ps)
# 控制DAC
dac=DAC(ps)
# 芯片, 读写操作, device_cfg
chip=CHIP(adc,dac,ps)

In [ ]:
chip.set_device_cfg(deviceType=1)
chip.set_read_mode(row=True)
chip.set_dac_read_V(0.1)
chip.adc.set_gain(3)
# i,num,bank, index, tia
res=chip.read_row_or_col(row_index=[1],col_index=[2,54,19,10,120,240,197,127,138])
print(res)

In [ ]:
for i in range(0,255,2):
    print(i+1,"\t",chip.numToBank_Index(i))

for i in range(1,256,2):
    print(i+1,"\t",chip.numToBank_Index(i))

In [ ]:
for i in range(0,255,2):
    print(i+1,"\t",adc.TIA_index_map(i,device=0,col=True))

for i in range(1,256,2):
    print(i+1,"\t",adc.TIA_index_map(i,device=0,col=True))

In [ ]:
print(adc.TIA_index_map(33))

### 测试indexMap

In [ ]:
x = 255
bank,index = numToBank_Index(x)
print(f"bank = {bank}, index = {index}")

In [ ]:
import cimCommand.singleCmdInfo as SCD

singleCmd=[]
for attribute_name in dir(SCD):
    attribute_value = getattr(SCD, attribute_name)
    if type(attribute_value)==dict:
        command_name=attribute_value.get("command_name",None)
        if command_name:
            singleCmd.append(attribute_value)
singleCmd.sort(key=lambda x: x['command_addr'])
for k,v in enumerate(singleCmd):
    # print(v)
    print(CMD(v))

In [ ]:
from CIMcommand import CMD,FAST_COMMAND_0,DAC_IN,DAC_CTR
from util import *
print(CMD(DAC_IN,command_data=(1<<24)|7))
print(CMD(DAC_CTR))

In [ ]:
data=0xF
data=set_zero(data,5)
print(bin(data))

data=0xF
data=set_one(data,5)
print(bin(data))

data=0xFFFF
data=place_bit(data,1,4,0b0000)
print(bin(data))

In [ ]:
from pc.commands import *
from pc.packets import *
from pc.ps import *

In [ ]:
cmd_list = [DAC_IN(0, 10), FAST_COMMAND_1(1)]
packet = Packet(0, cmd_list)
print(bytes(cmd_list[0]))
# print(bytes(packet))

# split_list = [1, 2, 3, 7, 8, 12]
# split_name_list = ['帧头', '模式', 'reg地址', 'reg值', 'cmd地址', 'cmd值']
# split_index = 0
# res_bin = ""
# for index, byte in enumerate(bytes(packet)):
#     res_bin += format(byte, '08b')
#     res_bin += "\t"
#     if index == split_list[split_index]:
#         print(split_name_list[split_index] + "\t\t" + res_bin)
#         res_bin = ""
#         split_index += 1

In [ ]:
ps = PS("192.168.1.10", 7)
